In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px

import plotly.io as pio
pio.renderers.default = "iframe"

from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

import pickle 

In [ ]:
pd.__version__

In [ ]:
df = pd.read_csv('/kaggle/input/engineering-placements-prediction/collegePlace.csv')

In [ ]:
df.shape

In [ ]:
df.head()
#df.tail()

In [ ]:
df.sample(4)

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.corr()['PlacedOrNot']

In [ ]:
# missing values
df.isnull().sum()

#drop null values if present
#df.dropna(inplace=True)

In [ ]:
# duplicate rows
print(df.duplicated().sum())

#drop duplicates
df.drop_duplicates(inplace=True)

In [ ]:
# Check if the duplicate rows are removed
print(df.duplicated().sum())

In [ ]:
#Plot the graph to visualize the output wrt major features
#plt.scatter(df['CGPA'],df['Internships'],c=df['PlacedOrNot']) 
fig = px.scatter(df, x="CGPA", y="Internships", color="PlacedOrNot",
                 hover_data=['CGPA'])
fig.show()

In [ ]:
fig = px.histogram(df, x='PlacedOrNot', color='PlacedOrNot', barmode='group')
fig.show()

In [ ]:
# Pie Chart
fig = px.pie(df, values=df['PlacedOrNot'].value_counts().values, names=df['PlacedOrNot'].value_counts().index, title='Placed Vs Not Placed')
fig.show()

In [ ]:
print("Max Age of Placed Person: ",df[(df['Age'] == df['Age'].max()) & (df['PlacedOrNot']==1)]['Age'].values[0])
print("Min Age of Placed Person: ",df[(df['Age'] == df['Age'].min()) & (df['PlacedOrNot']==1)]['Age'].values[0])

In [ ]:
print("Max Internships Done by the Placed Student: ",df[(df['Internships'] == df['Internships'].max()) & (df['PlacedOrNot']==1)]['Internships'].values[0])
print("No of students who did max Internships and are placed: ",df[(df['Internships'] == df['Internships'].max()) & (df['PlacedOrNot']==1)]['Internships'].value_counts().values[0])

print("Min Internships Done by the Placed Person: ",df[(df['Internships'] == df['Internships'].min()) & (df['PlacedOrNot']==1)]['Internships'].values[0])
print("No of students who did min Internships and are placed: ",df[(df['Internships'] == df['Internships'].min()) & (df['PlacedOrNot']==1)]['Internships'].value_counts().values[0])


In [ ]:
print("Max CGPA of Placed Student: ",df[(df['CGPA'] == df['CGPA'].max()) & (df['PlacedOrNot']==1)]['CGPA'].values[0])
print("No of students has max CGPA and are placed: ",df[(df['CGPA'] == df['CGPA'].max()) & (df['PlacedOrNot']==1)]['CGPA'].value_counts().values[0])

print("Min CGPA of Placed Person: ",df[(df['CGPA'] == df['CGPA'].min()) & (df['PlacedOrNot']==1)]['CGPA'].values[0])
print("No of students has min CGPA and are placed: ",df[(df['CGPA'] == df['CGPA'].min()) & (df['PlacedOrNot']==1)]['CGPA'].value_counts().values[0])


In [ ]:
fig = px.box(df, y='CGPA')
fig.show()

In [ ]:
fig = px.box(df, y='Age')
fig.show()

In [ ]:
fig = px.box(df, y=['Internships','CGPA', 'Age'])
fig.show()

In [ ]:
# convert Gender column to numeric
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

In [ ]:
df['Stream'].unique()

In [ ]:
# convert Stream column to numeric
df['Stream'] = df['Stream'].map({'Electronics And Communication': 1, 
                                 'Computer Science': 2,
                                'Information Technology': 3,
                                'Mechanical':4,
                                'Electrical':5,
                                'Civil':6})

In [ ]:
df.sample(5)

In [ ]:
# # you can reduce the features using PCA
# pca = PCA(n_components=2)
# X_pca = pca.fit_transform(X)
# X_pca_transform = pd.DataFrame(data=X_pca)
# #Plot the graph 
# plt.scatter(X_pca_transform[0],X_pca_transform[1],c=y)

In [ ]:
X = df.iloc[:,0:7]
y = df.iloc[:,-1]
X

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

## Logistic Regression

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
#using Logistic Regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

# Without Scaling 
clf.fit(X_train,y_train) 
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

#scaling has not much effect

## SGD Classifier

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier
#Using SGD Classifier
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(max_iter=1000, tol=1e-3)

# Without Scaling
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())


In [ ]:
from sklearn.linear_model import Perceptron
# this is same as SGDClassifier(loss="perceptron", eta0=1, learning_rate="constant", penalty=None)

clf = Perceptron(tol=1e-3, random_state=0)
# Without Scaling
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())


In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html#sklearn.linear_model.LogisticRegressionCV
# Using LogisticRegressionCV
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0)

# Without Scaling
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

## Decision Tree Classifier

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
# Using DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

#without scaling
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

## Random Forest Classifier

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=10, random_state=0)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

## Support Vector Machines

In [ ]:
#https://scikit-learn.org/stable/modules/svm.html

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
from sklearn.svm import SVC
#clf = SVC(gamma='auto')

svc = SVC()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
clf = GridSearchCV(svc, parameters)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Best Parameters:", clf.best_params_)
print("Best Score:", clf.best_score_)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("Best Parameters:", clf.best_params_)
print("Best Score:", clf.best_score_)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVC.html#sklearn.svm.NuSVC
from sklearn.svm import NuSVC
clf = NuSVC()

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, tol=1e-5)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

## Naive Bayes

In [ ]:
#https://scikit-learn.org/stable/modules/naive_bayes.html

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

#y_pred = gnb.fit(X_train, y_train).predict(X_test)
#print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())


In [ ]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

In [ ]:
from sklearn.naive_bayes import CategoricalNB
clf = CategoricalNB()

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
#scores = cross_val_score(clf, X_train, y_train, cv=5)
#print("Without Scaling and With CV: ",scores.mean())

## K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())


# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

In [ ]:
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("With CV: ",scores.mean())
print("Precision Score: ", precision_score(y_test, y_pred))
print("Recall Score: ", recall_score(y_test, y_pred))
print("F1 Score: ", f1_score(y_test, y_pred))


In [ ]:
param_grid = {
    'bootstrap': [False,True],
    'max_depth': [5,8,10, 20],
    'max_features': [3, 4, 5, None],
    'min_samples_split': [2, 10, 12],
    'n_estimators': [100, 200, 300]
}

rfc = RandomForestClassifier()

clf = GridSearchCV(estimator = rfc, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 1)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))
print(clf.best_params_)
print(clf.best_estimator_)

In [ ]:
clf = RandomForestClassifier(bootstrap=False, max_depth=5,max_features=None,
                             min_samples_split=2,
                             n_estimators=100, random_state=0)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("With CV: ",scores.mean())
print("Precision Score: ", precision_score(y_test, y_pred))
print("Recall Score: ", recall_score(y_test, y_pred))
print("F1 Score: ", f1_score(y_test, y_pred))

In [ ]:
pickle.dump(clf,open('model.pkl','wb'))

In [ ]:
model = pickle.load(open('model.pkl','rb'))

In [ ]:
age = 22
gender = 1 # 1=Male, 0=Female
stream = 3  # Electronics And Communication': 1, 
#              'Computer Science': 2,
#              'Information Technology': 3,
#              'Mechanical':4,
#              'Electrical':5,
#              'Civil':6
Internships = 2
CGPA = 6
Hostel = 1 # 1= stay in hostel, 0=not staying in hostel
HistoryOfBacklogs = 0 # 1 = had backlogs, 0=no backlogs

prediction = clf.predict([[age,gender,stream,Internships,CGPA,Hostel,HistoryOfBacklogs]])
prediction

In [ ]:
# Import all estimators list
from sklearn.utils import all_estimators


estimators = all_estimators(type_filter='classifier')
classification_estimators = []
i = 1
for name, class_ in estimators:
    classification_estimators.append(class_.__name__)
    print(f'{i}. {class_.__name__}')
    i += 1

In [ ]:
for name, class_ in estimators:
    module_name = str(class_).split("'")[1].split(".")[1]
    class_name = class_.__name__
    print(f'from sklearn.{module_name} import {class_name}')

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:

for estimator in classification_estimators:
    try:
        print("ALgorithm: ", estimator)
        clf = eval(estimator+'()')
        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        print("Without CV: ",accuracy_score(y_test,y_pred))
        scores = cross_val_score(clf, X_train, y_train, cv=10)
        print("With CV: ",scores.mean())
        print("Precision Score: ", precision_score(y_test, y_pred))
        print("Recall Score: ", recall_score(y_test, y_pred))
        print("F1 Score: ", f1_score(y_test, y_pred))
        print("____________________________________________________")
    except:
        print("Error")
    


In [ ]:
#for estimator in classification_estimators:
#    try:
#        print("ALgorithm: ", estimator)
#        clf = eval(estimator+'()')
#        clf.fit(X_train_scale,y_train)
#        y_pred = clf.predict(X_test_scale)
#        print("Without CV: ",accuracy_score(y_test,y_pred))
#        scores = cross_val_score(clf, X_train_scale, y_train, cv=5)
#        print("With CV: ",scores.mean())
#        print("Precision Score: ", precision_score(y_test, y_pred))
#        print("Recall Score: ", recall_score(y_test, y_pred))
#        print("F1 Score: ", f1_score(y_test, y_pred))
#        print("____________________________________________________")
#    except:
#        print("Error")

In [ ]:
#parameters = {
##    "loss":["deviance"],
#    "learning_rate": [0.05, 0.1, 0.15, 0.2],
#    "min_samples_split": np.linspace(0.1, 0.5, 12),
#    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
#    "max_depth":[8,10,12],
#    "max_features":["log2","sqrt"],
#    "criterion": ["friedman_mse"],
#    "subsample":[0.5,0.85, 0.9, 1.0],
#    "n_estimators":[10, 50, 80]
#    }

#clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=10, n_jobs=-1, verbose = 1)
#clf.fit(X_train,y_train)
#y_pred = clf.predict(X_test)
##print("Accuracy: ",accuracy_score(y_test,y_pred))
#print(clf.best_params_)
#print(clf.best_estimator_)

In [ ]:
#clf = GradientBoostingClassifier(learning_rate=0.2, max_depth=8, max_features='log2',
#                           min_samples_leaf=0.1, min_samples_split=0.1,
#                           n_estimators=50)
#clf.fit(X_train,y_train)
#y_pred = clf.predict(X_test)
#print("Without CV: ",accuracy_score(y_test,y_pred))
#scores = cross_val_score(clf, X_train, y_train, cv=10)
#print("With CV: ",scores.mean())
#print("Precision Score: ", precision_score(y_test, y_pred))
#print("Recall Score: ", recall_score(y_test, y_pred))
#print("F1 Score: ", f1_score(y_test, y_pred))